# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [1]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Using TensorFlow backend.


Loaded dataset with 25000 training samples, 25000 test samples


In [2]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 894, 25, 26, 200, 4, 2088, 7, 158, 5, 2437, 549, 18, 4, 1479, 675, 50, 26, 55, 171, 183, 8, 40, 133, 31, 42, 107, 411, 39, 2, 2, 628, 2, 75, 66, 144, 67, 90, 53, 5, 1284, 1881, 26, 1984, 5, 2, 2, 9, 17, 52, 17, 59, 70, 30, 1069, 41, 2, 21, 1038, 198, 12, 338, 114, 338, 116, 433, 487, 5, 2669, 4, 318, 302, 26, 2, 17, 76, 17, 4, 436, 904, 9, 35, 1032, 5, 373, 975, 4, 2681, 2267, 2, 1186, 26, 32, 1305, 394, 48, 75, 1133, 4, 206, 2, 73, 50, 26, 1297, 206, 844, 11, 49, 108, 15, 97, 25, 2, 5, 50, 26, 49, 15, 97, 25, 2, 14, 20, 64, 1367, 4, 1569, 243, 14, 889, 117, 317, 146, 38, 1264, 13, 122, 24, 990, 18, 14, 10, 10, 466, 101, 3401, 13, 70, 332, 5, 104, 38, 17, 13, 1829, 180, 8, 106, 13, 122, 24, 535, 233, 52, 13, 69, 57, 1398, 21, 16, 643, 3786, 170, 11, 246, 40, 6, 1374, 3526, 31, 566, 1533, 168, 245, 5, 60, 19, 57, 1398, 13, 16, 387, 180, 78, 24, 60, 2, 78, 45, 2, 819, 146, 55, 2, 7, 78, 102, 21, 14, 166, 1446, 3592, 3632, 63, 13, 423, 735, 1517, 10, 10, 57, 14, 418, 7,

Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [3]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

--- Review (with words) ---
['the', 'superb', 'have', 'he', 'may', 'of', 'logic', 'br', "didn't", 'to', 'drop', 'type', 'but', 'of', 'appeared', 'major', 'more', 'he', 'time', 'again', 'seems', 'in', 'just', 'scene', 'by', "it's", 'seen', 'dialogue', 'or', 'and', 'and', 'english', 'and', 'bad', 'had', 'real', 'can', 'made', 'up', 'to', 'remains', 'moon', 'he', 'psychological', 'to', 'and', 'and', 'it', 'movie', 'very', 'movie', 'would', 'well', 'at', 'cartoon', 'about', 'and', 'not', 'agree', 'series', 'that', 'death', 'little', 'death', 'love', 'entire', "you'll", 'to', 'reveals', 'of', 'excellent', 'instead', 'he', 'and', 'movie', 'get', 'movie', 'of', 'problem', 'directors', 'it', 'so', 'flat', 'to', 'tell', 'background', 'of', 'starred', 'unrealistic', 'and', 'trip', 'he', 'an', 'channel', 'getting', 'what', 'bad', 'recent', 'of', 'without', 'and', 'much', 'more', 'he', 'master', 'without', 'move', 'this', 'good', 'many', 'for', 'could', 'have', 'and', 'to', 'more', 'he', 'good', '

Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

...


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [4]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [5]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
embed_size = 32
hidden_size = 100

model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=embed_size))
model.add(LSTM(units=hidden_size, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

...

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [6]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [7]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 3

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]  # first batch_size samples
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]  # rest for training

# TODO: Train your model
model.fit(X_train2, y_train2,
          batch_size=batch_size,
          epochs=num_epochs,
         validation_data=(X_valid, y_valid))

Train on 24936 samples, validate on 64 samples
Epoch 1/3
24936/24936 [==============================] - 546s 22ms/step - loss: 0.4572 - acc: 0.7787 - val_loss: 0.4116 - val_acc: 0.8438
Epoch 2/3
24936/24936 [==============================] - 545s 22ms/step - loss: 0.2797 - acc: 0.8900 - val_loss: 0.3395 - val_acc: 0.7969
Epoch 3/3
24936/24936 [==============================] - 546s 22ms/step - loss: 0.2309 - acc: 0.9115 - val_loss: 0.3333 - val_acc: 0.8750


In [10]:
import os
# Save your model, so that you can quickly load it in future (and perhaps resume training)
cache_dir='./'
model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join(cache_dir, model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [11]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.87484


#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

...

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!